In [1]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import sklearn
import warnings
import statsmodels.api as sm
import matplotlib.patches as mpatches
from matplotlib import pyplot as plt
from scipy import stats
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)
%matplotlib inline
sns.set_style('white')

In [2]:
crime = pd.read_excel("Validate Regression.xls", encoding = "ISO-8859-1", skiprows=4) 

In [3]:
crime.columns

Index(['City', 'Population', 'Violent\ncrime',
       'Murder and\nnonnegligent\nmanslaughter',
       'Rape\n(revised\ndefinition)1', 'Rape\n(legacy\ndefinition)2',
       'Robbery', 'Aggravated\nassault', 'Property\ncrime', 'Burglary',
       'Larceny-\ntheft', 'Motor\nvehicle\ntheft', 'Arson'],
      dtype='object')

In [4]:
crimechange = crime.rename(index=str, columns={'Violent\ncrime': 'ViolentCrime', 
                        'Murder and\nnonnegligent\nmanslaughter': 'Murder',
                       'Rape\n(revised\ndefinition)1' : 'Rape',
                       'Aggravated\nassault' : 'AggravatedAssault',
                       'Property\ncrime': 'PropertyCrime',
                       'Larceny-\ntheft': 'LarcenyTheft',
                       'Motor\nvehicle\ntheft': 'MotorVehicleTheft',
                       'Arson3': 'Arson'})
crimechange = crimechange.drop(['Rape\n(legacy\ndefinition)2'], axis=1)
crimechange = crimechange.drop(['City'], axis=1)
crimechange['Murder?'] = np.where(crimechange['Murder']>0, 1, 0)
crimechange.head(10)

,Population,ViolentCrime,Murder,Rape,Robbery,AggravatedAssault,PropertyCrime,Burglary,LarcenyTheft,MotorVehicleTheft,Arson,Murder?
0,5463.0,74.0,1.0,NaN,0.0,72.0,217.0,8.0,202.0,7.0,0.0,1
1,55710.0,49.0,1.0,NaN,17.0,30.0,1328.0,334.0,925.0,69.0,8.0,1
2,39577.0,76.0,0.0,NaN,25.0,51.0,1716.0,363.0,1252.0,101.0,11.0,0
3,10942.0,33.0,0.0,NaN,0.0,31.0,259.0,57.0,185.0,17.0,0.0,0
4,50058.0,229.0,5.0,NaN,57.0,155.0,1975.0,474.0,1415.0,86.0,37.0,1
5,248718.0,575.0,2.0,NaN,162.0,359.0,5900.0,993.0,4628.0,279.0,77.0,1
6,10850.0,50.0,1.0,NaN,2.0,46.0,207.0,34.0,163.0,10.0,0.0,1
7,4090.0,5.0,0.0,NaN,0.0,4.0,51.0,19.0,32.0,0.0,1.0,0
8,3499.0,11.0,0.0,NaN,0.0,11.0,42.0,8.0,31.0,3.0,0.0,0
9,11820.0,66.0,0.0,NaN,13.0,48.0,662.0,144.0,488.0,30.0,4.0,0


In [5]:
crimechange.isnull().any()

Population            True
ViolentCrime          True
Murder                True
Rape                  True
Robbery               True
AggravatedAssault     True
PropertyCrime         True
Burglary              True
LarcenyTheft          True
MotorVehicleTheft     True
Arson                 True
Murder?              False
dtype: bool

In [6]:
crimechange['Population']=crimechange['Population'].fillna(0)
crimechange['ViolentCrime']=crimechange['ViolentCrime'].fillna(0)
crimechange['Murder']=crimechange['Murder'].fillna(0)
crimechange['Rape']=crimechange['Rape'].fillna(0)
crimechange['Robbery']=crimechange['Robbery'].fillna(0)
crimechange['AggravatedAssault']=crimechange['AggravatedAssault'].fillna(0)
crimechange['PropertyCrime']=crimechange['PropertyCrime'].fillna(0)
crimechange['Burglary']=crimechange['Burglary'].fillna(0)
crimechange['LarcenyTheft']=crimechange['LarcenyTheft'].fillna(0)
crimechange['MotorVehicleTheft']=crimechange['MotorVehicleTheft'].fillna(0)
crimechange['Arson']=crimechange['Arson'].fillna(0)

In [7]:
stats.chisqprob = lambda chisq, crimechange: stats.chi2.sf(chisq, crimechange)

lr = LogisticRegression(C=1e9)
Y = crimechange['Murder?']
X = crimechange[['Population','Robbery','Arson']]

# Declare predictors.
X_statsmod = crimechange[['Population','Robbery','Arson']]

X_statsmod['intercept'] = 1 

logit_model=sm.Logit(Y,X)
result=logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.579261
         Iterations 11
                           Logit Regression Results                           
Dep. Variable:                Murder?   No. Observations:                   66
Model:                          Logit   Df Residuals:                       63
Method:                           MLE   Df Model:                            2
Date:                Thu, 17 Jan 2019   Pseudo R-squ.:                  0.1502
Time:                        22:04:57   Log-Likelihood:                -38.231
converged:                       True   LL-Null:                       -44.987
                                        LLR p-value:                  0.001164
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Population -9.815e-06   1.29e-05     -0.758      0.448   -3.52e-05    1.56e-05
Robbery        0.0437      0

c:\users\ben\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=0)
lreg = LogisticRegression()
lreg.fit(X_train, Y_train)

y_pred = lreg.predict(X_test)
print('The accuracy of logistic regression classifier for this data set {:.3f}.'.format(lreg.score(X_test, Y_test)))

The accuracy of logistic regression classifier for this data set 0.412.


c:\users\ben\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [9]:
KFold = model_selection.KFold(n_splits=5, random_state=5)
model = LogisticRegression()
scoring = 'accuracy'
results = model_selection.cross_val_score(model, X_train, Y_train, cv=KFold, scoring=scoring)
print("The accuracy rate for the 10-fold cross is %.3f." % (results.mean()))

The accuracy rate for the 10-fold cross is 0.487.


c:\users\ben\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\ben\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\ben\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\ben\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\ben\appdata\local\programs\python\python37-32\l

In [10]:
trainsize = int(crimechange.shape[0]*0.75)  
crimechange_test = crimechange.iloc[trainsize:, :].copy() 
crimechange_train = crimechange.iloc[:trainsize, :].copy()

regr = linear_model.LinearRegression()
YTrain = crimechange_train['Murder?'].values.reshape(-1, 1)
XTrain = crimechange_train[['Population','Robbery','Arson']]
regr.fit(XTrain, YTrain)
print('\nThe R-squared simple model is:')
print(regr.score(XTrain, YTrain))

origparams = np.append(regr.coef_, regr.intercept_) 


crimechange_train['PopulationArson'] = crimechange_train['Population'] * crimechange_train['Arson']
crimechange_train['PopulationRobbery'] = crimechange_train['Population'] * crimechange_train['Robbery']
crimechange_train['PopulationRoot'] = crimechange_train['Population'] ** 0.5
crimechange_train['PopulationSquared'] = crimechange_train['Population'] ** 2
crimechange_train['PopulationCubed'] = crimechange_train['Population'] ** 3
crimechange_train['RobberyArson'] = crimechange_train['Robbery'] * crimechange_train['Arson']
crimechange_train['RobberyRoot'] = crimechange_train['Robbery'] ** 0.5
crimechange_train['RobberySquared'] = crimechange_train['Robbery'] ** 2
crimechange_train['RobberyCubed'] = crimechange_train['Robbery'] ** 3
crimechange_train['ArsonRoot'] = crimechange_train['Arson'] ** 0.5
crimechange_train['ArsonSquared'] = crimechange_train['Arson'] ** 2
crimechange_train['ArsonCubed'] = crimechange_train['Arson'] ** 3
                             
regrBig = linear_model.LinearRegression()
X_train2 = crimechange_train[['Population','Robbery','Arson',
                     'PopulationArson',
                     'PopulationRobbery','RobberyArson',
                     'ArsonRoot','ArsonSquared','ArsonCubed',
                     'RobberyRoot','RobberySquared','RobberyCubed',
                     'PopulationRoot','PopulationSquared','PopulationCubed'
                    ]]
regrBig.fit(X_train2, YTrain)
print('\nThe R-squared complex model is:')
print(regrBig.score(X_train2, YTrain))

newparams = np.append(
    regrBig.coef_[0,0:(len(origparams)-1)],
    regrBig.intercept_)

print('\nSmall vs. Large Model:')
compare = np.column_stack((origparams, newparams))
prettycompare = np.array2string(
    compare,
    formatter={'float_kind':'{0:.4f}'.format})
print(prettycompare)


The R-squared simple model is:
0.28994253697780137

The R-squared complex model is:
0.4575615340398438

Small vs. Large Model:
[[0.0000 0.0000]
 [-0.0018 0.0000]
 [0.0044 0.0000]
 [0.2721 0.0645]]


In [11]:
YTest = crimechange_test['Murder?'].values.reshape(-1, 1)
XTest = crimechange_test[['Population','Robbery','Arson']]
print('\nThe R-squared simple model is:')
print(regr.score(XTest, YTest))

crimechange_test['PopulationArson'] = crimechange_test['Population'] * crimechange_test['Arson']
crimechange_test['PopulationRobbery'] = crimechange_test['Population'] * crimechange_test['Robbery']
crimechange_test['PopulationRoot'] = crimechange_test['Population'] ** 0.5
crimechange_test['PopulationSquared'] = crimechange_test['Population'] ** 2
crimechange_test['PopulationCubed'] = crimechange_test['Population'] ** 3
crimechange_test['RobberyArson'] = crimechange_test['Robbery'] * crimechange_test['Arson']
crimechange_test['RobberyRoot'] = crimechange_test['Robbery'] ** 0.5
crimechange_test['RobberySquared'] = crimechange_test['Robbery'] ** 2
crimechange_test['RobberyCubed'] = crimechange_test['Robbery'] ** 3
crimechange_test['ArsonRoot'] = crimechange_test['Arson'] ** 0.5
crimechange_test['ArsonSquared'] = crimechange_test['Arson'] ** 2
crimechange_test['ArsonCubed'] = crimechange_test['Arson'] ** 3
                             
X_test2 = crimechange_test[['Population','Robbery','Arson',
                     'PopulationArson',
                     'PopulationRobbery','RobberyArson',
                     'ArsonRoot','ArsonSquared','ArsonCubed',
                     'RobberyRoot','RobberySquared','RobberyCubed',
                     'PopulationRoot','PopulationSquared','PopulationCubed'
                    ]]
print('\nThe R-squared complex model is:')
print(regrBig.score(X_test2, YTest))


The R-squared simple model is:
0.19894778638045396

The R-squared complex model is:
-31.652159006687974


In [12]:
ridgeregr = linear_model.Ridge(alpha=10, fit_intercept=False)
ridgeregr.fit(X_train, Y_train)
print(ridgeregr.score(X_train, Y_train))
origparams = ridgeregr.coef_[0]
print(origparams)

ridgeregrBig = linear_model.Ridge(alpha=10, fit_intercept=False)
ridgeregrBig.fit(X_train2, Y_train)
print(ridgeregrBig.score(X_train2, Y_train))
print('\nSmall vs Large Model Estimates')
prettycompare = np.array2string(compare,
                               formatter = {'float_kind':'{0:.3f}'.format})
print(prettycompare)

0.016245858706122385
5.319494512195654e-06
0.14770483410049995

Small vs Large Model Estimates
[[0.000 0.000]
 [-0.002 0.000]
 [0.004 0.000]
 [0.272 0.065]]


c:\users\ben\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number9.115294e-37
  overwrite_a=True).T


In [13]:
print(ridgeregr.score(XTest, YTest))
print(ridgeregrBig.score(X_test2, YTest))

0.008459246300217882
-49.33362629993892


In [14]:
lass = linear_model.Lasso(alpha=10)
lassfit = lass.fit(X_train, Y_train)
print('R² for the model with few features:')
print(lass.score(X_train, Y_train))
origparams = np.append(lassfit.coef_, lassfit.intercept_)
print('\nParameter estimates for the model with few features:')
print(origparams)

lassBig = linear_model.Lasso(alpha=10)
lassBig.fit(X_train2, Y_train)
print('\nR² for the model with many features:')
print(lassBig.score(X_train2, Y_train))
origparams = np.append(lassBig.coef_, lassBig.intercept_)
print('\nParameter estimates for the model with many features:')
print(origparams)

R² for the model with few features:
0.22346172049431112

Parameter estimates for the model with few features:
[ 2.43137890e-06 -8.31675969e-04  0.00000000e+00  3.53336848e-01]

R² for the model with many features:
0.09374654423262718

Parameter estimates for the model with many features:
[-4.45449348e-08 -0.00000000e+00 -0.00000000e+00 -8.99369715e-09
  9.09939586e-11 -3.24604575e-06 -0.00000000e+00 -7.45901116e-05
  6.20107086e-07 -0.00000000e+00 -6.06192700e-07  3.02224149e-11
 -0.00000000e+00  7.10943626e-12 -3.17310725e-18  4.41399917e-01]


c:\users\ben\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [15]:
print(lass.score(XTest, YTest))
print(lassBig.score(X_test2, YTest))

0.13080683832282336
-0.025974332409875123
